In [1]:
# импорты
import nltk
import re
import pickle
from string import punctuation
import math
from tqdm import tqdm_notebook
import csv
import gensim

import pymystem3
m = pymystem3.Mystem() #для использования лемматизации

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

C:\Users\Aleksandra\AppData\Roaming\Python\Python36\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def unpack(data):
    input = open(data, 'rb')
    obj = pickle.load(input)
    input.close()
    return obj

# загружаем частоты лем униграмм
unigrams = unpack('1stemgrams.data')
# обрезали по частоте 20, а еще убрали пробел из начала слов
unigrams = {w[1:]:f for w,f in unigrams.items()}

In [3]:
# читает из файла, убирает двойные пробелы и ручные переносы, последний \n
def reading(file):
    f = open('texts\\{}.txt'.format(file), 'r', encoding='utf-8')
    text = f.read()
    text = text.replace('  ', ' ')
    text = text.replace('-\n', '')
    if text[-1] == '\n': # убираем последний \n, если такой есть 
        text = text[:-1]
    f.close()
    return text

In [4]:
def minimummaker(): #ф-ция превращения текстового файла с минимумом в питоновский список. Запаковка списка
    with open('min.txt', encoding='utf-8') as file:
        lemtokens = [morph.parse(i)[0].normal_form for i in re.findall('\w+', file.read().lower())] #делаем список лемм слов
        minimum = list(set(lemtokens)) #убираем повторы
        output = open('minimum.pkl', 'wb')
        pickle.dump(minimum, output, 2)
        output.close()

def loadminimum(): #распаковка cписка с минимумом. Возвращает неупорядоченный список
    input = open('minimum.pkl', 'rb')
    minimum = pickle.load(input)
    input.close()
    return minimum

#minimummaker()
minimum = loadminimum()
#print(minimum[-50:]) #проверка работы
#print(len(minimum)) #2549 слов в списке

In [5]:
# загрузка модели
def model_loading(file):
    model = gensim.models.KeyedVectors.load_word2vec_format(file, binary=False)
    model.init_sims(replace=True)
    print('Done!') 
    return model

model = model_loading('news_upos_cbow_600_2_2018.vec') #на загрузку тратится минут 10

Done!


In [6]:
# загрузка словаря ASIS
with open('syns.data', 'rb') as f:
     asis = pickle.load(f)

In [7]:
# Таблица конверсии в UPoS из тэгов Mystem
# словарь, переводящий теги mystem в universal теги моделей
mystem_tags = {'A' : 'ADJ',
       'ADV' : 'ADV',
       'ADVPRO' : 'ADV',
       'ANUM' : 'ADJ',
       'APRO' : 'DET',
       'COM' : 'ADJ',
       'CONJ' : 'SCONJ',
       'INTJ' : 'INTJ',
       'NONLEX' : 'X',
       'NUM' : 'NUM',
       'PART' : 'PART',
       'PR' : 'ADP',
       'S' : 'NOUN',
       'SPRO' : 'PRON',
       'UNKN' : 'X',
       'V' : 'VERB',
       'X' : 'X',
      'PROPN' : 'PROPN'} #последних 2 тегов в майстеме нет, но они задаются в классе для слов в соответсвующими пометами

# словарь, переводящий теги пайморфи в universal теги моделей
pymorphy_tags = {'ADJF':'ADJ',
    'ADJS' : 'ADJ',
    'ADVB' : 'ADV',
    'COMP' : 'ADV',
    'GRND' : 'VERB',
    'INFN' : 'VERB',
    'NOUN' : 'NOUN',
    'PRED' : 'ADV',
    'PRTF' : 'ADJ',
    'PRTS' : 'VERB',
    'VERB' : 'VERB'}

In [9]:
class Token():
    def __init__(self, w):
        
        self.num = None # номер в тексте
        self.complexity = None # сложность слова
        
        # три варианта инициализации: 
        ## из анализа текста, 
        ## из уже имеющегося объекта (для дочернего класса ComplexWord) 
        ## из строки
        
        if isinstance(w, dict): # если получили результат работы mystem
            
            self.text = w['text']  # сам токен
            self.len = len(w['text']) # его длина
            
            # определяет, сделан ли анализ и, соответственно, рассматривать ли как слово, требующее упрощения
            gram = w.get('analysis')
            if gram:
                self.lexem = gram[0]['lex']  # лемма
                
                if not self.named_entity(gram[0]):  # именованная сущность или нет
                    self.pos = self.pos_tag(gram[0]['gr'])  # часть речи
                else:
                    self.pos = 'PROPN' # universal tag for named entity - у майстема таких нет
                
                    
            elif any(p in w['text'] for p in punctuation+'–«»'): # если это знак пунктуации (может быть с пробелом!)
                self.lexem = '_PUNKTUATION_'
                self.pos = None
            
            elif not re.findall('\S',w['text']): # если это только пробельные символы
                self.lexem = '_SPACE_'
                self.pos = None
                
            # остальное - неизвестная и ненужная ерунда?
            else:
                self.lexem = '_UNK_'
                self.pos = 'UNKN' # mystem tag for unknown
            
            
        elif isinstance(w, Token): # для определения объектов дочернего класса ComplexWord
            self.text = w.text
            self.num = w.num
            self.lexem = w.lexem 
            self.len = w.len
            self.pos = w.pos
            self.complexity = w.complexity
            
        
        elif isinstance(w, str): # если хотим как класс токен определить строку, полученную из словаря или модели
            self.text = w
            self.pos = None
            self.lexem = w
            self.len = len(w)
            self.num = None
            self.complexity = None
            
            '''                
            else:
                self.text = w # задаём саму строку текстом токена
                self.pos = tagging(w) # определяем тег с помощью пайморфи, переводя при этом в universal'''
        
    # вытаскивает часть речи из разбора майстем
    def pos_tag(self,gram):
        if ',' in gram:
            gram = gram.split(',')[0]
        if '=' in gram:
            gram = gram.split('=')[0]
        return gram
        
    # определяет по тегам, является ли именованной сущностью
    def named_entity(self,gram):
        markers = {'сокр': ' - сокращение', 'фам': ' - фамилия', 'имя': ' - имя собственное', 'гео': ' - название места', }
        if any(m in gram['gr'] for m in markers.keys()):
            return True
        else:
            return False

    def complexity_params(self, param = 'freq'):
        # если по частотности
        if param == 'freq':
            self.complexity = unigrams.get(self.lem, 0)
            
        # если по коэффициенту информативности. Отрицательное значение. Чем он меньше, тем сложнее
        elif param == 'inf':
            self.complexity = math.log((unigrams.get(self.lexem, 0)+1)/(sum(f for f in unigrams.values())+1))
        
    
    def is_complex(self, threshold = '600', use_min = False, len_threshold = 1000):
        exceptions = ['_PUNKTUATION_', '_SPACE_', '_UNK_']
        # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
        if not any(exception in self.lexem for exception in exceptions) and self.pos not in ['PROPN']:
            
            # если показатель сложности - вхождение в минимум
            if use_min:
                if self.lexem not in minimum:
                    return True
                else:
                    return False

            # если показатель сложности - пороговое значение сложности
            # также может использоваться длина. По умолчанию слишком большое - 1000 (т.е. этот параметр не учитывается)
            else:
                if self.complexity < threshold or self.len > len_threshold:
                    return True
                else:
                    return False
        else:
            return False
    
    def convert_universal(self):
        if self.pos in mystem_tags:
            self.pos = mystem_tags[self.pos]
        else:
            self.pos = 'X' # Х - universal тег для неизвестных слов
        
'''Пример вывода атрибутов объектов класса 
token.num, token.text, token.lem, token.pos, token.complexity, token.is_complex(threshold, use_min):

223 Кукушкин _NAMED_ENTITY_ S -19.06687873346149 False
224   _SPACE_ None -19.06687873346149 False
225 является являться V -7.502177090710664 False
226   _SPACE_ None -19.06687873346149 False
227 должником должник S -11.184186527172464 True
228   _SPACE_ None -19.06687873346149 False
229 банка банк S -9.300701131889967 False
230 .  _PUNKTUATION_ None -19.06687873346149 False
'''

'Пример вывода атрибутов объектов класса \ntoken.num, token.text, token.lem, token.pos, token.complexity, token.is_complex(threshold, use_min):\n\n223 Кукушкин _NAMED_ENTITY_ S -19.06687873346149 False\n224   _SPACE_ None -19.06687873346149 False\n225 является являться V -7.502177090710664 False\n226   _SPACE_ None -19.06687873346149 False\n227 должником должник S -11.184186527172464 True\n228   _SPACE_ None -19.06687873346149 False\n229 банка банк S -9.300701131889967 False\n230 .  _PUNKTUATION_ None -19.06687873346149 False\n'

In [10]:
'''# вытаскивает часть речи из разбора майстем
def pos(gram):
    if ',' in gram:
        gram = gram.split(',')[0]
    if '=' in gram:
        gram = gram.split('=')[0]
    return gram
'''

#анализ текста 
def text_structuring(text, param, threshold, use_min):
    # анализирует текст 
    analysis = m.analyze(text)
    tokens = []
    for i, w in enumerate(analysis): # состаляем список объектов Tokens
        token = Token(w)
        token.num = i # добавляем токену в атрибуты его номер в тексте
        token.complexity_params(param) # переопределяем сложность на основе выбранного параметра
        print(token.num, token.text, token.lexem, token.pos, token.complexity, token.is_complex(threshold, use_min))
        tokens.append(token)
    return tokens

In [11]:
# ПАРАМЕТРЫ анализа слов
complexity_type = 'inf'
global_threshold = -8.5
use_min=True

In [56]:
text = reading('news1')
tokens = text_structuring(text, complexity_type, global_threshold, use_min)

0 Экс экс S -12.407584813777852 True
1 - _PUNKTUATION_ None -19.06687873346149 False
2 депутат депутат S -9.544944468987213 False
3   _SPACE_ None -19.06687873346149 False
4 из из PR -5.430146815838169 False
5   _SPACE_ None -19.06687873346149 False
6 Астрахани астрахань PROPN -11.590972764094092 False
7   _SPACE_ None -19.06687873346149 False
8 обманул обманывать V -9.722269740680273 True
9   _SPACE_ None -19.06687873346149 False
10 ВТБ втб PROPN -14.313288542355124 False
11   _SPACE_ None -19.06687873346149 False
12 на на PR -4.174175661002731 False
13   _SPACE_ None -19.06687873346149 False
14 750 _UNK_ UNKN -19.06687873346149 False
15   _SPACE_ None -19.06687873346149 False
16 млн _UNK_ UNKN -19.06687873346149 False
17 ,  _PUNKTUATION_ None -19.06687873346149 False
18 переписав переписывать V -11.046279583564518 True
19   _SPACE_ None -19.06687873346149 False
20 имущество имущество S -9.643525957370555 True
21 
 _SPACE_ None -19.06687873346149 False
22 
 _SPACE_ None -19.0668787334

179 рублей рубль S -8.4421182633064 False
180 .  _PUNKTUATION_ None -19.06687873346149 False
181 В в PR -3.486155131943022 False
182   _SPACE_ None -19.06687873346149 False
183 МВД мвд PROPN -10.974639326737279 False
184   _SPACE_ None -19.06687873346149 False
185 России россия PROPN -7.264452850232696 False
186   _SPACE_ None -19.06687873346149 False
187 по по PR -5.1820044394556914 False
188   _SPACE_ None -19.06687873346149 False
189 Астраханской астраханский A -11.87544940342511 True
190   _SPACE_ None -19.06687873346149 False
191 области область S -8.188095962474803 False
192   _SPACE_ None -19.06687873346149 False
193 возбудили возбуждать V -9.872871016106204 True
194   _SPACE_ None -19.06687873346149 False
195 уголовное уголовный A -9.858339983431934 True
196   _SPACE_ None -19.06687873346149 False
197 дело дело S -6.454827778200301 False
198   _SPACE_ None -19.06687873346149 False
199 по по PR -5.1820044394556914 False
200   _SPACE_ None -19.06687873346149 False
201 факту факт 

In [33]:
class Substitution(Token):
    def __init__(self, w):
        super().__init__(w)
        self.similarity = None
        self.fitness = None
    
    # для слов из словаря и тезауруса: определяем тег пайморфи, переводим в формат universal - так быстрее, чем майстемом
    def tagging(self, w):
        tag = morph.parse(w)[0].tag.POS
        if tag in pymorphy_tags:
            return pymorphy_tags[tag]
        else:
            return 'X' # Х - universal тег для неизвестных слов

    # в случае, когда Substitution получено не через модель, и его близость неизвестна
    def measuse_similarity(self, target):
        #print(self.lexem, target.lexem, self.pos, target.pos)
        subst_query = str(self.lexem+'_'+self.pos)
        target_query = str(target.lexem+'_'+target.pos)
        if subst_query in model and target_query in model:
            self.similarity = model.similarity(subst_query, target_query)
        # может понадобиться return self
    
    # приписываем атрибуты словам, взятым из словаря или тезауруса
    def setting_atr(self, target):
        self.pos = self.tagging(self.lexem)
        self.complexity_params(complexity_type)
        self.measuse_similarity(target)
        
    def language_model(self):
        pass

In [42]:
class Complex_word(Token):
    def __init__(self, w):
        super().__init__(w)
        self.substituts = None
        
    # список замен, в зависимости от выбранной базы
    def search_substituts(self, base_type='model'):
        
        # поиск по модели
        def model_search(lexem, pos):
            query = str(lexem+'_'+pos)
            #print(query)
            if query in model:
                # формируем список квазисинонимов той же части речи
                # при этом превращаем их в объекты соответствующего класса
                syn_tokens = []
                for syn, sim in model.most_similar(positive=[query]):
                    syn_text = syn[:syn.find('_')] # текст до части речи
                    
                    syn_tok = Substitution(syn_text) # из текстовой строки инициализируем объект класса
                    syn_tok.pos = syn[syn_tok.len+1:] # часть речи
                    
                    syn_tok.complexity_params(complexity_type) # сложность по функции в зависимости от выбранного параметра
                    
                    syn_tok.similarity = sim # а близость по параметру модели
                    
                    syn_tokens.append(syn_tok)
                    
                return syn_tokens
            else:
                return [] 

        # поиск по YARN
        def yarn_search(target, filepath = 'yarn-synsets1.csv'):
            with open(filepath, "r", newline="") as file: # постепенный просмотр файла с синсетами (множествами синонимов)
                reader = csv.DictReader(file, delimiter=';')
                lst = []
                for i,row in enumerate(reader):
                    cur_line = row['words'].split(';') # считываем колонку с синсетами
                    if len(cur_line)>1:
                        if target.lexem in cur_line:
                            del(reader)
                            for c in cur_line:
                                if ' ' not in c and c!=target.lexem: # формируем список однословных синонимов
                                    sub_tok = Substitution(c)
                                    
                                    sub_tok.setting_atr(target)
                                    
                                    lst.append(sub_tok) 
                            #TODO: выделить неоднословные в отдельный класс и поискать их частотность по n-граммам?
                            break
                #print(lst)
                return lst 
        
        #поиск по ASIS
        def asis_search(target):
            if target.lexem in asis:
                lst = []
                for s in asis[target.lexem]:
                    if ' ' not in s: # формируем список однословных синонимов
                        sub_tok = Substitution(s)
                        sub_tok.setting_atr(target)
                        lst.append(sub_tok)
                return lst
            else:
                return []

        
        if base_type == 'model':
            self.substituts = model_search(self.lexem, self.pos)
            
        if base_type == 'yarn':
            self.substituts = yarn_search(self)
        
        if base_type == 'asis':
            self.substituts = asis_search(self)
        
        return self

In [57]:
# Отбор сложных слов, превращение их в подкласс сложных слов и поиск замен
# несложные слова также остаются на своих местах
def selecting_complex(tokens, base_type='asis'):
    complex_words = tokens[:]
    for token in tokens:
        if token.is_complex(global_threshold, use_min):
            comp_token = Complex_word(token) # токен становится Сложным словом
            comp_token.convert_universal() # превращаем теги майстем в теги universal (чтоб искать сходство для словарных синонимов)
            comp_token.search_substituts(base_type=base_type)
            complex_words.append(comp_token)
            # код для принтов
            if comp_token.substituts:
                for syn in comp_token.substituts:
                    print (comp_token.lexem, base_type, ':', syn.lexem, syn.pos, syn.complexity, syn.similarity)
        else:
            complex_words.append(token)
    return complex_words

complex_words = selecting_complex(tokens, 'model')

экс model : бывший ADJ -8.677760295793927 0.7683254480361938
экс model : тогдашний ADJ -10.103334441464746 0.5065518021583557
экс model : бытность NOUN -12.239249498958637 0.4892158508300781
экс model : екс NOUN -19.06687873346149 0.4332670569419861
экс model : опальный ADJ -12.767929486605547 0.4332367181777954
экс model : калининград::александр::ярошук PROPN -19.06687873346149 0.41629523038864136
экс model : новоназначенный ADJ -16.501929375999953 0.41382887959480286
экс model : владивосток::игорь::пушкарев PROPN -19.06687873346149 0.39897608757019043
экс model : самара::дмитрий::азаров PROPN -19.06687873346149 0.39640069007873535
экс model : алексей::лопатин PROPN -19.06687873346149 0.38368552923202515
обманывать model : доверчивый ADJ -11.767081366703328 0.5600028038024902
обманывать model : облапошивать VERB -14.534279240308233 0.5213795304298401
обманывать model : обман NOUN -10.519156337010429 0.5006035566329956
обманывать model : обирать VERB -12.03602125734537 0.46529370546340

версия model : анзор::шулай PROPN -19.06687873346149 0.3669460117816925
версия model : сентябрясрок NOUN -19.06687873346149 0.3618030846118927
версия model : поддержывать ADV -19.06687873346149 0.3547150492668152
версия model : выяснить VERB -19.06687873346149 0.3528914451599121
версия model : алексей::совет PROPN -19.06687873346149 0.3519218862056732
версия model : установить VERB -19.06687873346149 0.35140955448150635
версия model : версією X -19.06687873346149 0.34945371747016907
версия model : спецверсия NOUN -19.06687873346149 0.3330376148223877
следствие model : следователь NOUN -10.020234454156096 0.4980217516422272
следствие model : обвиняемый NOUN -11.40582235109966 0.3772863745689392
следствие model : расследование NOUN -10.945992712368652 0.37061572074890137
следствие model : подсудимый NOUN -11.04825326841574 0.3528651297092438
следствие model : разбирательство NOUN -11.617962630917289 0.34993213415145874
следствие model : подсудимая NOUN -14.056243439365234 0.3364616632461

развлекательный model : розважальний ADJ -19.06687873346149 0.49025487899780273
развлекательный model : досугов NOUN -19.06687873346149 0.46511679887771606
развлекательный model : культурно ADV -11.98097726909588 0.4544321894645691
развлекательный model : научно-познавательный ADJ -17.6805843723416 0.44929102063179016
развлекательный model : торгово-выставочный ADJ -19.06687873346149 0.4462418258190155
развлекательный model : розважальный ADJ -19.06687873346149 0.4423186779022217
комплекс model : комлекс NOUN -19.06687873346149 0.5466686487197876
комплекс model : комплексі X -19.06687873346149 0.5284945964813232
комплекс model : кластер NOUN -12.668283798926282 0.40543022751808167
комплекс model : комплексів X -19.06687873346149 0.3905414342880249
комплекс model : объект NOUN -8.989395857877883 0.3888908922672272
комплекс model : инфраструктура NOUN -11.352201259660562 0.3674769103527069
комплекс model : зрк X -19.06687873346149 0.35963672399520874
комплекс model : туркомплекс NOUN -19

сумма model : 184-фз NOUN -19.06687873346149 0.34243690967559814
сумма model : госказна NOUN -16.022356295738067 0.3422086834907532
сумма model : наличные NOUN -12.042229703007854 0.34196174144744873
сумма model : единоразовый ADJ -19.06687873346149 0.3418538570404053
задолженность model : долг NOUN -9.234511313428486 0.6632111072540283
задолженность model : недоимка NOUN -12.423089000313817 0.5910729169845581
задолженность model : задолжность NOUN -19.06687873346149 0.5510221719741821
задолженность model : кредиторский ADJ -13.498534229700393 0.4623718857765198
задолженность model : переплата NOUN -15.305678617767928 0.4603768587112427
задолженность model : дебиторский ADJ -13.819854661301003 0.45859596133232117
задолженность model : просрочивать VERB -12.96432013884792 0.4239703118801117
задолженность model : невыплаченный ADJ -15.57037117199501 0.4114803671836853
задолженность model : задолжать VERB -13.129342528379063 0.38337722420692444
задолженность model : неплатеж NOUN -13.6869

должник model : алиментщица NOUN -19.06687873346149 0.4301639497280121
должник model : злостный ADJ -12.625932192828568 0.4247032105922699
должник model : залогодатель NOUN -13.879492927620735 0.4178668260574341
должник model : алиментный ADJ -16.176506975565324 0.40190696716308594
настоящий model : мечтный ADJ -19.06687873346149 0.5820115804672241
настоящий model : незапамятный ADJ -12.596079229678887 0.576244592666626
настоящий model : приостанавливаются NOUN -19.06687873346149 0.5649785399436951
настоящий model : 4,965 NUM -19.06687873346149 0.5413105487823486
настоящий model : 2003,51 NUM -19.06687873346149 0.5342518091201782
настоящий model : 1,235 NUM -19.06687873346149 0.5290502309799194
настоящий model : utc PROPN -19.06687873346149 0.5218778848648071
настоящий model : 5,281 NUM -19.06687873346149 0.5206108093261719
настоящий model : золотореченск PROPN -19.06687873346149 0.5202372074127197
настоящий model : 1,518 NUM -19.06687873346149 0.5135911703109741
взыскивать model : взы

### теперь, когда для каждого сложного слова есть список его синонимов с нужными параметрами, можно делать модель на н-граммах
### контекстное окно: 10 слов (по 5 слева и справа, можно задавать?)

In [58]:
def make_window(token, window):
    context = [token]
    left_ind = token.num-1
    right_ind = token.num+1
    # добавляем по одному слову слева и/или справа, пока не наберется window + само слово
    while len(context)<window+1:
        while left_ind >= 0:
            left_w = tokens[left_ind]
            
            # проверка, что это слово
            if any(exception in left_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_']):
                left_ind+=1
            else:
                context[:0] = left_w #вставляем слово слева от цепочки
                break
                
        while right_ind < len(tokens):
            right_w = tokens[right_ind]
            
            # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
            if any(exception in right_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_']):
                right_ind+=1
            else:
                context.append(right_w) # справа от цепочки
                break
            
for token in complex_words:
    if isinstance(token, Complex_word):
        print(token.text)
        context = make_window(token, 10)
        print(context)

Экс
None
обманул


TypeError: can only assign an iterable